<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/2021/2%20Advanced/330Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 写真の減色

画像データは1600万色程度の色彩を表現できるが、これは一般的な写真の画素数よりも多い。つまり、全く使っていない色はたくさんある。また、次の写真のように、カラー写真とはいっても、実際に使っている色はそれほど多くないように見えるものもある。

![](https://live.staticflickr.com/8380/8640855620_102dda223f_z_d.jpg)

(CC BY 2.0 2009 SteFou! via Flickr)

この写真を、できるだけ少ない色数で表示してみよう。

## 白黒の場合

まず、この画像を入手し、色彩を落として、numpy arrayの形にする。414 x 640W pixelの画像なので、サイズ(414,640)の実数のarrayの各要素がそれぞれの画素の明るさを表す。


In [ ]:
from imageio import imread
import PIL

img = imread("https://live.staticflickr.com/8380/8640855620_102dda223f_z_d.jpg")
img.shape

画像の画素ごとのデータは、通常はR(赤),G(緑),B(青)それぞれ0〜255の256段階で表現される。3色を平均して白黒画像にする。

In [ ]:
# RGB方向の平均をとり、255で割る。
import numpy as np

#平均値は実数になってしまうので、8ビット負号なし整数(0〜255)に変換しておく。
gray = np.average(img, axis=2).astype(np.uint8)
gray.shape

In [ ]:
gray

In [ ]:
# displayはJupyterの機能。
display(PIL.Image.fromarray(gray))

ヒストグラムを作る。

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
Y,X = np.histogram(gray, bins=32)
plt.plot(X[:-1],Y)

ピクセルの明るさが、100〜220に集中しているのがわかります。

## 単純に階調を8等分して、8階調にする

何も考えず、この画像を8階調に落してみます。0〜255の階調を8階調にするには、
* 0〜31を全部0に
* 32〜63を全部32に
* ...
* 224〜255を全部224に
置きかえればいいのです。

これにはLUT(Lookup table)を使います。

まず、0〜255を8つの色に対応させる対応表(lookup table)を準備します。

In [ ]:
lut = np.zeros(256, dtype=np.uint8)  #8ビット整数の配列

for i in range(256):
    lut[i] = (i//32)*32  #整数32で割ってから32倍する。

plt.xlabel("original grades")
plt.ylabel("stepping grades")
plt.plot(lut)

numpyのFancy indexでグレースケール画像を段階画像に一発変換。

In [ ]:
simple8 = lut[gray]
display(PIL.Image.fromarray(simple8))

なんかのっぺりとしてわかりにくい絵になってしまいました。コントラストが小さいせいでしょう。灰色の階調をもう少しこまかくとりたいところです。

## ヒストグラムにもとづいた彩色



ヒストグラムにより、暗い色が少なく、明るい色に偏っていることがわかっているので、明るいところを細かい階調で表現するのがよさそうです。

そこで、8階調に落とす時に、各階調の画素の数が均等になるようにしましょう。つまり、上のグラフを、面積が等しくなるように8等分します。

そのために、まずすべての画素を暗い順に並べます。

In [ ]:
height, width = gray.shape
Npix = height*width
# 1次元にして、輝度の小さい順にソートする。
pixels = np.sort(gray.reshape(Npix))
pixels

In [ ]:
plt.xlabel("Pixel")
plt.ylabel("Brightness")
plt.plot(pixels)
for i in range(9):
    plt.axvline(Npix*i//8, 0, 255)

In [ ]:
# (Npix/8) 個目の画素は?
pixels[Npix//8]

なので、輝度が108以下の画素は全部輝度を108/2=54にする。

Npix 番目から Npix/4 番目の画素は、

In [ ]:
pixels[Npix//4]

なので、108以上128以下の画素は全部輝度を(108+128)/2=118にする。

以下同様。これを8階調まとめてやってみよう。

In [ ]:
for i in range(8):
    # range
    smallest = Npix*i//8
    largest  = Npix*(i+1)//8 - 1
    # brightness of the pixels at the two ends
    Ps = pixels[smallest]
    Pl = pixels[largest]
    # average of the two
    Pm = (Ps+Pl)/2
    print(i,Ps,Pl,Pm)

うまいぐあいにできている。これを使って、対応表を作ってみよう。

In [ ]:
for i in range(8):
    # range
    smallest = Npix*i//8
    largest  = Npix*(i+1)//8 - 1
    # brightness of the pixels at the two ends
    Ps = int(pixels[smallest])
    Pl = int(pixels[largest])
    # average of the two
    Pm = (Ps+Pl)//2
    for j in range(Ps, Pl):
        print(j,Pm)
        lut[j] = Pm

plt.plot(lut)

またFancy Indexで色変換。

In [ ]:
equi8=lut[gray]
display(PIL.Image.fromarray(equi8))

ちょっとメリハリがついて、背景と桜が見分けられるようになった。

## カラーの場合

デジタルカラー画像は赤、緑、青それぞれの強度が256段階あり、1600万通りの色彩がありうる。RGBをそれぞれ8段階にしたとしても、64色は必要になる。8色にまで落とすためには、3色をon/offの2段階にまで落とす必要があり、やる前からうまくいかないのは目に見えている。

In [ ]:
simple = img.copy()
# 画素の輝度が128より小さい点はすべて0にする。
simple[img<128]=0
# 128より大きい点はすべて255にする。
simple[img>=128]=255
display(PIL.Image.fromarray(simple))

酷い。

もとの絵に含まれているピクセルの色の分布を、RGB3次元の空間で表してみる。

In [ ]:
# 写真に使われていた色を、RGB空間にプロットする。
import plotly.graph_objs as go
import cv2

tiny = cv2.resize(img, (img.shape[1]//4, img.shape[0]//4))
height, width = tiny.shape[:2]
Npix = height*width

pixels = tiny.reshape(Npix, 3)

colors = ['rgb({0},{1},{2})'.format(r,g,b) for r,g,b in pixels[:]]
trace=dict(type='scatter3d',
           x= pixels[:,0],
           y= pixels[:,1],
           z= pixels[:,2],
           mode='markers',
           marker=dict(color=colors,
                       size=3)
          )
fig = go.Figure(data=trace)
fig.update_layout(scene = dict(
                    xaxis_title='R',
                    yaxis_title='G',
                    zaxis_title='B'))
fig.show()

使われている色はとても偏っていることがわかる。

そこで、k-平均分類という機械学習の手法を使ってみる。k-平均分類は、機械学習において、多次元空間に散在する多数の点を、近いもの同士で集めて、クラスターにする手法である。

桜の写真の画素は、R,G,Bを3つの軸とする立方体の中の点で表される。上の写真には640x414=265000点の画素があり、それらは立方体の中で偏って分布している。おそらくピンクに相当する部分に多数の点が集中し、ほかに緑や黒に相当する領域に小さな集団を作っているはずだ。

そこで、この3次元空間内の点を、近いものどうしをつないでいくことで、領域分割する手法がk-平均法 (k-means classifier)である。

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_kmeans_digits_001.png)

2次元でのk-近傍分類器はこんな感じ。10グループに分けろ、と指定すると、10種類に分けてくれる。それぞれのクラスターの重心点で色を代表させれば、色数を減らせる。

k平均法は、機械学習ライブラリscikit-learnに含まれているKMeansを使う。

In [ ]:
# Pythonデータサイエンスハンドブック 5.11.1
from sklearn.cluster import KMeans

height, width = img.shape[:2]
Npix = height*width

#画像をピクセル列に変換する(そうしないとk-meansが使えない)
pixels = img.reshape(Npix, 3)

# 8つの代表的な色をさがさせる。
kmeans = KMeans(n_clusters=8, max_iter=2000)
kmeans.fit(pixels)
kmeans.cluster_centers_

代表色をならべる

In [ ]:
import numpy as np

squares = np.zeros([8,100,100,3])
for i in range(8):
    squares[i,:,:] = kmeans.cluster_centers_[i]
    display(PIL.Image.fromarray(squares[i].astype(np.uint8)))

In [ ]:
# それぞれのピクセルに一番近い中心は何番か。
kmeans.predict(pixels)

In [ ]:
# ピクセルごとの色の変換表を作る

new_pixels = kmeans.cluster_centers_[kmeans.predict(pixels)]
new_pixels

In [ ]:
# new_pixelsを8ビット整数にし、arrayの形をもとに戻し、画像として表示する。
display(PIL.Image.fromarray(new_pixels.astype(np.uint8).reshape(height, width, 3)))

たった8色でもここまで雰囲気が出せました。ただし、平均的な色でクラスターを代表するせいで、発色が落ちています。

どのようにクラスター化されたかを、RGB3次元で見てみましょう。

In [ ]:
# 各ピクセルの色が、どの色に減色されたかを散布図で示す。8つの領域に分割されている。

import plotly.graph_objs as go

height, width = tiny.shape[:2]
Npix = height*width

pixels = tiny.reshape(Npix, 3)
pred   = kmeans.cluster_centers_[kmeans.predict(pixels)].reshape(Npix, 3)

colors = ['rgb({0},{1},{2})'.format(r,g,b) for r,g,b in pred[:]]
trace=dict(type='scatter3d',
           x= pixels[:,0],
           y= pixels[:,1],
           z= pixels[:,2],
           mode='markers',
           marker=dict(color=colors,
                       size=3)
          )
fig = go.Figure(data=trace)
fig.update_layout(scene = dict(
                    xaxis_title='R',
                    yaxis_title='G',
                    zaxis_title='B'))
fig.show()

機械学習を利用するときは、こういった検証作業をきっちりやっておくと、不可解な現象が起こるのを防げます。

## 以下は準備中

さらに色の縞を減らすために、誤差拡散法によるディザリングを行います。

Ditheringの性能を向上するためには、色空間内でパレットに含まれる点が作る凸包ができるだけ多くの点をふくみ、なおかつピクセル色から最近パレット色までの距離が小さくなるようにパレット色を選ぶ必要がある。上の手法では、前者の条件がないために、発色が劣化する。

これは幾何学的な最適化問題だが、時間をかけて良いなら解決できる。

凸包の中の点は、混色で必ず近似できるが、凸包の外の点(色域外点)はいかに混色しても作れない。そのため、外の点のほうがペナルティを大きくする必要がある。色域外点は、凸包の表面の最近点からの距離が遠いほどペナルティが大きい。

In [ ]:
# すべての点をべつべつに評価するのは大変なので、あらかじめ16x16x16のヒストグラムにおさめておく。

H = np.histogramdd(img.reshape(width*height,3) / 255, bins=(16,16,16))
hist = H[0].reshape([16*16*16])

fp = np.array([[x,y,z]
               for x in (0,0.999)
               for y in (0,0.999)
               for z in (0,0.999)])
fp = np.random.random([8,3])*0.1+0.9

fp

In [ ]:
# 使われている色を、頻度順に並べてみる。

ind=[(x,y,z)
     for x in range(16)
     for y in range(16)
     for z in range(16)]
x=0
y=0
pal=np.zeros([160,160,3])
for xyz in sorted(ind, key=lambda x: H[0][x], reverse=True):
    if H[0][xyz] > 0:
        pal[y*10:(y+1)*10,x*10:(x+1)*10] = np.array(xyz)*255/15
        x += 1
        if x == 16:
            x=0
            y += 1
#        print(xyz, H[0][xyz])
display(PIL.Image.fromarray(pal.astype(np.uint8)))

# histogtamddの格納順がよくわからないので、実験する

data = np.array([[0,0,0], [1,0,0], [0,1,0], [0,0,1], [0.2, 0.4, 0.8]])
H = np.histogramdd(data, bins=(4,4,4))
H[0][0,1,3]

(x,y,z)の順で対応していることがわかった。

In [ ]:
?np.histogramdd

In [ ]:
from scipy.spatial import ConvexHull
hull =  ConvexHull(fp)

In [ ]:
# Distances of points from a 3D convex hull
#
# https://stackoverflow.com/questions/55460133/distance-to-convex-hull-from-point-in-3d-in-python

from PyGEL3D import gel

def dist(hull, points):
    # Construct PyGEL Manifold from the convex hull
    m = gel.Manifold()
    for s in hull.simplices:
        m.add_face(hull.points[s])

    dist = gel.MeshDistance(m)
    res = []
    for p in points:
        # Get the distance to the point
        # But don't trust its sign, because of possible
        # wrong orientation of mesh face
        d = dist.signed_distance(p)

        # Correct the sign with ray inside test
        if dist.ray_inside_test(p):
            if d > 0:
                d *= -1
        else:
            if d < 0:
                d *= -1
        res.append(d)
    return np.array(res)


In [ ]:
points = np.array([(x+0.5,y+0.5,z+0.5)
                   for x in range(16)
                   for y in range(16)
                   for z in range(16)])
points /= 16

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.plot(dist(hull,points))

ほとんどの点がhullの外にある。

ペナルティを計算する。

* hull内の点は、一番近い点からの距離をペナルティとする。
* hull外の点は、hullまでの距離をペナルティとする。
* 両者のバランスは係数Cで調節する。

In [ ]:
import random

def Penalty(fp,points,hist):
    C=250.0
    hull =  ConvexHull(fp)
    distances = dist(hull, points)
    penalty = 0.0
    insider = 0
    outsider = 0
    for d,p,w in zip(distances, points, hist):
        if w > 0.0:
            if d > 0:
                penalty += d*C*w
                outsider += 1
            else:
                Dmin = 1e99
                for v in fp:
                    D = np.linalg.norm(v-p)
                    if D < Dmin:
                        Dmin = D
                penalty += Dmin*w
                insider += 1
    return penalty, insider, outsider

def Penalty2(fp,points,hist):
    """
    Numpy style
    """
    C=250.0
    hull =  ConvexHull(fp)
    distances = dist(hull, points)
    penalty = 0.0
    insider = 0
    outsider = 0
    d = distances[hist>0]
    p = points[hist>0]
    w = hist[hist>0]
    
    outsiders = d[d>0]
    penalty = C*np.sum(outsiders*w[d>0])

    insiders = p[d<0]
    Dmins = np.array([np.min(np.linalg.norm(fp-P, axis=1))
                      for P in insiders])
    penalty += np.sum(Dmins*w[d<0])
    return penalty, insiders.shape[0], outsiders.shape[0]


def show_palette(fp):
    Ncolor = fp.shape[0]
    squares = np.zeros([20,20*Ncolor,3])
    for i in range(Ncolor):
        squares[:,i*20:(i+1)*20] = fp[i]*255
    display(PIL.Image.fromarray(squares.astype(np.uint8)))

In [ ]:
P, ins, outs = Penalty(fp,points,hist)
lasti=0
for i in range(30000):
    particle = i % 8
    fp_new = fp.copy()
    fp_new[particle] += (np.random.random([3])-0.5)*0.25
    fp_new[particle] = np.clip(fp_new[particle],0.,1.)
    # print(fp_new[particle])
    P_new, ins, outs = Penalty2(fp_new, points, hist)
    if P_new < P:
        fp = fp_new
        P = P_new
        if lasti + 20 < i:
            print(i,P,ins, outs)
            show_palette(fp)
            lasti = i

In [ ]:
def dither2(img, palettes):
    accum = np.zeros(3)
    result = np.zeros_like(img)
    for r,row in enumerate(img):
        for c,pixel in enumerate(row):
            accum += pixel
            bestpal = np.zeros(3)
            bestD   = 1e99
            for pal in palettes:
                D = accum - pal
                D = np.dot(D,D)
                if D < bestD:
                    bestD = D
                    bestpal = pal
            accum -= bestpal
            result[r,c] = bestpal
    return result

In [ ]:
def dither3(img, palettes):
    accum = np.zeros(3)
    result = np.zeros_like(img)
    for r,row in enumerate(img):
        for c,pixel in enumerate(row):
            accum += pixel
            
            D = palettes - accum
            D = np.sum(D*D, axis=1)
            bestpal=np.argmin(D)
            accum -= palettes[bestpal]
            result[r,c] = palettes[bestpal]
    return result

In [ ]:
dithered = dither3(img, fp*255)
display(PIL.Image.fromarray(dithered.astype(np.uint8)))

In [ ]:
display(PIL.Image.fromarray(img))


In [ ]:
fp_new = np.zeros([20,3])
fp_new[:8] = fp
fp = fp_new
fp.shape


In [ ]:
fp[8:16]=fp[:8]

In [ ]:
# 16 colors

P, ins, outs = Penalty2(fp,points,hist)
lasti=0
for i in range(30000):
    Ncolor = fp.shape[0]
    particle = i % Ncolor
    fp_new = fp.copy()
    fp_new[particle] += (np.random.random([3])-0.5)*0.25
    fp_new[particle] = np.clip(fp_new[particle],0.,1.)
    # print(fp_new[particle])
    P_new, ins, outs = Penalty2(fp_new, points, hist)
    if P_new < P:
        fp = fp_new
        P = P_new
        if lasti + 20 < i:
            print(i,P,ins, outs)
            show_palette(fp)
            lasti = i

In [ ]:
dithered = dither3(img, fp*255)
display(PIL.Image.fromarray(dithered.astype(np.uint8)))